# Option Pricing Surace

## Table of Contents
* [1)Benchmark](#first-bullet)
* [2) LSM](#second-bullet)
* [2) Dynamic Chebyshev](#third-bullet)
* [4) Error Surface](#fourth-bullet)

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import os

# local
from DC import DynamicChebyshev
from LSM import MonteCarloOptionPricing
from FD import AmericanPutFiniteDifference

In [ ]:
# Parameter inputs
S0 = 100  # current stock price
r = 0.03  # risk-free rate
K = 100.   # strike price
λ = 0.4 # intensity rate of the Poisson process
dim = 100_000  # number of simulation paths
n = 252  # number of exercise rights per year
n_chebyshev_pol = 100 # degree of chebyshev polynomials
seed = 15_001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression
M = 1_000 # number of price steps for Finite Differnece
N = 40_000 # number of time steps for Finite Difference 
σ = 0.25 # volatiltiy parameter in diffusion term

In [ ]:
# Define maturities and strikes
maturities = np.array([1/12, 2/12, 3/12, 6/12, 9/12, 1, 15/12, 18/12, 2, 30/12, 3, 4])
strikes = np.arange(0.8 * S0, 1.21 * S0, 0.05 * S0)

## 1) Benchmark <a class="anchor" id="first-bullet"></a>

In [ ]:
# Lists to store results
prices_fd = []
maturity_list_fd = []
strike_list_fd = []

In [ ]:
# Iterate over each maturity and strike combination
for T in maturities:
    for K in strikes:
        Implicit_Finite_Difference = AmericanPutFiniteDifference(K, r, M)
        FD = Implicit_Finite_Difference.implicit_FD(S0, σ, T, N)
        # Store the results
        prices_fd.append(FD)
        maturity_list_fd.append(T)
        strike_list_fd.append(K)
        # Print
        print(f'K/T: {K}/{T}, F.D.: {FD:.4f}')

K/T: 80.0/0.08333333333333333, F.D.: 0.0016
K/T: 85.0/0.08333333333333333, F.D.: 0.0254
K/T: 90.0/0.08333333333333333, F.D.: 0.2034
K/T: 95.0/0.08333333333333333, F.D.: 0.9323
K/T: 100.0/0.08333333333333333, F.D.: 2.7672
K/T: 105.0/0.08333333333333333, F.D.: 5.9470
K/T: 110.0/0.08333333333333333, F.D.: 10.1848
K/T: 115.0/0.08333333333333333, F.D.: 15.0012
K/T: 120.0/0.08333333333333333, F.D.: 20.0000
K/T: 80.0/0.16666666666666666, F.D.: 0.0403
K/T: 85.0/0.16666666666666666, F.D.: 0.1985
K/T: 90.0/0.16666666666666666, F.D.: 0.6910
K/T: 95.0/0.16666666666666666, F.D.: 1.8213
K/T: 100.0/0.16666666666666666, F.D.: 3.8498
K/T: 105.0/0.16666666666666666, F.D.: 6.8529
K/T: 110.0/0.16666666666666666, F.D.: 10.7019
K/T: 115.0/0.16666666666666666, F.D.: 15.1611
K/T: 120.0/0.16666666666666666, F.D.: 20.0019
K/T: 80.0/0.25, F.D.: 0.1424
K/T: 85.0/0.25, F.D.: 0.4618
K/T: 90.0/0.25, F.D.: 1.1885
K/T: 95.0/0.25, F.D.: 2.5363
K/T: 100.0/0.25, F.D.: 4.6556
K/T: 105.0/0.25, F.D.: 7.5793
K/T: 110.0/0.25,

In [ ]:
# Convert lists to numpy arrays for processing
maturities_np_fd = np.array(maturity_list_fd)
strikes_np_fd = np.array(strike_list_fd)
prices_np_fd = np.array(prices_fd)

# Creating a grid for maturities and strikes
maturity_grid, strike_grid = np.meshgrid(np.linspace(maturities_np_fd.min(), maturities_np_fd.max(), 100),
                                         np.linspace(strikes_np_fd.min(), strikes_np_fd.max(), 100))

# Interpolating the prices onto the grid

price_grid = griddata((maturities_np_fd, strikes_np_fd), prices_np_fd, (maturity_grid, strike_grid), method='cubic')

In [ ]:
# Creating the plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plotting the surface
surface = ax.plot_surface(maturity_grid, strike_grid, price_grid, cmap='viridis', edgecolor='none')

# Adding labels and title
ax.set_xlabel('Maturity (Years)')
ax.set_ylabel('Strike Price')
ax.set_zlabel('Option Price')
plt.title('Surface Plot of Interpolated Prices')

# Adding a color bar for reference
plt.colorbar(surface, shrink=0.5, aspect=5)

# Display the plot
plt.show()

## 2) LSM <a class="anchor" id="second-bullet"></a>

In [ ]:
# Lists to store results
prices_lsm = []
maturity_list_lsm = []
strike_list_lsm = []

# Iterate over each maturity and strike combination
for T in maturities:
    for K in strikes:
        # Initialize the Monte Carlo pricer
        mcpricer = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed)
        # Simulate the asset paths
        mcpricer.GeometricBrownianMotion_vec()
        # Price the option using LSM
        option_price = mcpricer.american_option_LSM(poly_degree, otype='put')[0]

        # Store the results
        prices_lsm.append(option_price)
        maturity_list_lsm.append(T)
        strike_list_lsm.append(K)
        print(f'K/T: {K}/{T}, LSM: {option_price:.4f}')

In [ ]:
# Convert lists to numpy arrays for processing
maturities_np_lsm = np.array(maturity_list_lsm)
strikes_np_lsm = np.array(strike_list_lsm)
prices_np_lsm = np.array(prices_lsm)

# Creating a grid for maturities and strikes
maturity_grid, strike_grid = np.meshgrid(np.linspace(maturities_np_lsm.min(), maturities_np_lsm.max(), 100),
                                         np.linspace(strikes_np_lsm.min(), strikes_np_lsm.max(), 100))

# Interpolating the prices onto the grid

price_grid = griddata((maturities_np_lsm, strikes_np_lsm), prices_np_lsm, (maturity_grid, strike_grid), method='cubic')

In [ ]:
# Creating the plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plotting the surface
surface = ax.plot_surface(maturity_grid, strike_grid, price_grid, cmap='viridis', edgecolor='none')

# Adding labels and title
ax.set_xlabel('Maturity (Years)')
ax.set_ylabel('Strike Price')
ax.set_zlabel('Option Price')
plt.title('Black Scholes - LSM')

# Adding a color bar for reference
plt.colorbar(surface, shrink=0.5, aspect=5)

# Display the plot
plt.show()

## 3) Dynamic Chebyshev <a class="anchor" id="third-bullet"></a>

In [ ]:
# Lists to store results
prices_dc = []
maturity_list_dc = []
strike_list_dc = []

# Iterate over each maturity and strike combination
for T in maturities:
    for K in strikes:
        DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, 32, n_chebyshev_pol, seed, use_AV)
        # a) offline phase
        domain = DC_option_pricing.calculate_truncated_domain_GBM()
        xknots = DC_option_pricing.calculate_nodal_points(domain)
        BS = DC_option_pricing.generate_GBM_path(xknots)
        Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

        # b) online phase
        DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)[0]

        # Store the results
        prices_dc.append(DC)
        maturity_list_dc.append(T)
        strike_list_dc.append(K)
        print(f'K/T: {K}/{T}, D.C.: {DC:.4f}')

In [ ]:
# Convert lists to numpy arrays for processing
maturities_np_dc = np.array(maturity_list_dc)
strikes_np_dc = np.array(strike_list_dc)
prices_np_dc = np.array(prices_dc)

# Creating a grid for maturities and strikes
maturity_grid, strike_grid = np.meshgrid(np.linspace(maturities_np_dc.min(), maturities_np_dc.max(), 100),
                                         np.linspace(strikes_np_dc.min(), strikes_np_dc.max(), 100))

# Interpolating the prices onto the grid

price_grid = griddata((maturities_np_dc, strikes_np_dc), prices_np_dc, (maturity_grid, strike_grid), method='cubic')

In [ ]:
# Creating the plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plotting the surface
surface = ax.plot_surface(maturity_grid, strike_grid, price_grid, cmap='viridis', edgecolor='none')

# Adding labels and title
ax.set_xlabel('Maturity (Years)')
ax.set_ylabel('Strike Price')
ax.set_zlabel('Option Price')
plt.title('Dynamic Chebyshev - LSM')

# Adding a color bar for reference
plt.colorbar(surface, shrink=0.5, aspect=5)

# Display the plot
plt.show()

## 4) Error Surface <a class="anchor" id="fourth-bullet"></a>